<a href="https://colab.research.google.com/github/ashleyssssss/wildfire-prediction/blob/main/Modification_Loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install rioxarray
import numpy as np
import random
import os
import rasterio
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

data = np.load("/content/drive/MyDrive/NEW_MODIS_Combined/new_data.npz")
X = data["X"]
y = data["y"]

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 88.7 MB/s eta 0:00:00


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Conv2DTranspose, Concatenate, ZeroPadding2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation,
    UpSampling2D, Concatenate, ZeroPadding2D, Layer
)
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50V2

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

class ResizeLayer(Layer):
    def call(self, input):
        input_tensor, target_tensor = input
        target_height = tf.shape(target_tensor)[1]
        target_width = tf.shape(target_tensor)[2]
        return tf.image.resize(input_tensor, [target_height, target_width])

def decoder_block(input_tensor, skip_tensor, num_filters):
    x = ResizeLayer()([input_tensor, skip_tensor])
    x = Concatenate()([x, skip_tensor])
    x = conv_block(x, num_filters)
    return x

def build_resnetv2_unet(input_shape=(128, 128, 3)):
    inputs = Input(input_shape)

    """ Pre-trained ResNetV2 Encoder """
    base_model = ResNet50V2(include_top=False, weights="imagenet", input_tensor=inputs)

    # Encoder feature maps (you can verify with model.summary())
    s1 = base_model.get_layer("conv1_conv").output       # 64x64
    s2 = base_model.get_layer("conv2_block3_out").output # 32x32
    s3 = base_model.get_layer("conv3_block4_out").output # 16x16
    s4 = base_model.get_layer("conv4_block6_out").output # 8x8
    b1 = base_model.get_layer("conv5_block3_out").output # 4x4 (bridge)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)  # 4→8
    d2 = decoder_block(d1, s3, 256)  # 8→16
    d3 = decoder_block(d2, s2, 128)  # 16→32
    d4 = decoder_block(d3, s1, 64)   # 32→64

    # Final upsampling to match input 128x128
    x = UpSampling2D((2, 2))(d4)     # 64→128
    x = Conv2D(32, 3, padding="same", activation="relu")(x)

    # outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(x)
    outputs = Conv2D(4, 1, padding="same", activation="softmax")(x)

    model = Model(inputs, outputs, name="ResNetV2_U-Net")
    return model


In [ ]:
input_shape = X.shape[1:]
model = build_resnetv2_unet(input_shape)

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import SparseCategoricalCrossentropy
import tensorflow.keras.backend as K

def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        y_true = tf.one_hot(y_true, depth=4)  # Convert to one-hot
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())

        # Calculate focal loss
        cross_entropy = -y_true * K.log(y_pred)
        weight = alpha * K.pow(1. - y_pred, gamma)
        focal_loss = weight * cross_entropy

        return K.mean(focal_loss)
    return loss

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    # y_true: integer labels (sparse) [batch, H, W]
    # y_pred: probabilities [batch, H, W, num_classes]
    y_true = K.cast(y_true, 'int32')
    y_true_one_hot = K.one_hot(y_true, num_classes=4)  # Convert to one-hot [batch, H, W, 4]
    y_pred = K.cast(y_pred, 'float32')

    # Compute intersection and union for each class
    intersection = K.sum(y_true_one_hot * y_pred, axis=[1, 2])
    union = K.sum(y_true_one_hot, axis=[1, 2]) + K.sum(y_pred, axis=[1, 2])

    # Dice score per class, averaged over batch
    dice = K.mean((2. * intersection + smooth) / (union + smooth), axis=0)
    return K.mean(dice)  # Average over classes


def hybrid_loss(alpha=0.5):
    """
    Combines categorical focal loss and dice loss.
    alpha: weight for focal loss (1 - alpha for dice loss)
    """
    fl = focal_loss(alpha=0.25, gamma=2.0)  # You can tune these if needed

    def loss(y_true, y_pred):
        focal = fl(y_true, y_pred)
        dice = 1.0 - dice_coefficient(y_true, y_pred)  # Convert dice score to loss
        return alpha * focal + (1 - alpha) * dice

    return loss

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

In [ ]:
from keras.metrics import MeanIoU
class MeanIoUMetric(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes, name='mean_iou', **kwargs):
        super().__init__(num_classes=num_classes, name=name, **kwargs)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        # No need to squeeze y_true if it's already shape (batch, H, W)
        return super().update_state(y_true, y_pred, sample_weight)

model.compile(optimizer='adam',
              loss=hybrid_loss(alpha=0.7),
              metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=5, callbacks=callbacks)

Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 756s 16s/step - accuracy: 0.8794 - loss: 0.2409 - learning_rate: 0.0010
Epoch 2/5


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate
  current = self.get_monitor_value(logs)


44/44 ━━━━━━━━━━━━━━━━━━━━ 740s 16s/step - accuracy: 0.9998 - loss: 0.2251 - learning_rate: 0.0010
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 711s 16s/step - accuracy: 0.9999 - loss: 0.2251 - learning_rate: 0.0010
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 707s 16s/step - accuracy: 0.9999 - loss: 0.2250 - learning_rate: 0.0010
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 705s 16s/step - accuracy: 0.9999 - loss: 0.2250 - learning_rate: 0.0010
